In [0]:
# Configuration
catalog = "cpg_industry"
gold_sales = "gold_sales"
silver_schema = "silver"

In [0]:
# Drop schema
# spark.sql(f"DROP SCHEMA IF EXISTS {catalog}.{gold_sales} CASCADE")
# spark.sql(f"DROP TABLE IF EXISTS {catalog}.{gold_sales}.b2c_overall_kpis")

In [0]:
# Create schema
spark.sql(f"CREATE SCHEMA IF NOT EXISTS {catalog}.{gold_sales}")

### B2C Overall Kpis

In [0]:
# B2C Overall KPIs
spark.sql(f"""
CREATE OR REPLACE TABLE {catalog}.{gold_sales}.b2c_overall_kpis AS
WITH base AS (
    SELECT
        coi.order_item_id,
        coi.product_id,
        coi.quantity,
        coi.unit_price AS selling_price,
        coi.unit_price - p.unit_price AS margin,
        coi.unit_price * coi.quantity AS total_amount,
        ((coi.unit_price - p.unit_price) / coi.unit_price) * 100 AS profit_margin,
        coi.order_id AS s_oid,
        p.product_name,
        p.department,
        p.category,
        p.brand,
        p.retail_price,
        p.unit_price AS cost_price,
        p.release_date,
        co.order_id AS c_oid,
        co.order_date AS ordered_date,
        co.order_status,
        co.total_amount AS total_amount_order,
        co.payment_method AS payment_method_usage,
        c.consumer_id,
        c.name AS consumer_name,
        c.gender,
        c.age,
        c.registration_date,
        c.is_active,
        c.city,
        c.state,
        c.country,
        i.location_name,
        i.inventory_id,
        i.quantity_on_hand,
        i.inventory_status,
        i.reorder_level
    FROM {catalog}.{silver_schema}.cpg_consumer_order_items coi
    LEFT JOIN {catalog}.{silver_schema}.cpg_consumer_order co ON coi.order_id = co.order_id
    LEFT JOIN {catalog}.{silver_schema}.cpg_consumer c ON co.consumer_id = c.consumer_id
    LEFT JOIN {catalog}.{silver_schema}.cpg_product p ON coi.product_id = p.product_id
    LEFT JOIN {catalog}.{silver_schema}.cpg_inventory i ON p.product_id = i.product_id
),
total_metrics AS (
    SELECT
        COALESCE(SUM(total_amount), 0) AS total_consumer_sales,
        COALESCE(AVG(total_amount), 0) AS average_order_value,
        COALESCE(COUNT(DISTINCT order_id), 0) AS total_order_count
    FROM {catalog}.{silver_schema}.cpg_consumer_order
),
order_frequency AS (
    SELECT COALESCE(AVG(order_count), 0) AS order_frequency
    FROM (
        SELECT consumer_id, COUNT(order_id) AS order_count
        FROM {catalog}.{silver_schema}.cpg_consumer_order
        GROUP BY consumer_id
    ) sub
),
active_consumers AS (
    SELECT COALESCE(COUNT(DISTINCT consumer_id), 0) AS active_consumer_count
    FROM {catalog}.{silver_schema}.cpg_consumer
    WHERE is_active = TRUE
),
orders_sales AS (
    SELECT
        consumer_id,
        COUNT(DISTINCT c_oid) AS total_orders,
        SUM(total_amount) AS total_sales_revenue,
        ROUND(SUM(total_amount) / NULLIF(COUNT(DISTINCT c_oid), 0)) AS avg_order_value
    FROM base
    GROUP BY consumer_id
),
sales_category AS (
    SELECT consumer_id, category, SUM(total_amount) AS sales_by_category
    FROM base
    GROUP BY consumer_id, category
),
sales_brand AS (
    SELECT consumer_id, brand, SUM(total_amount) AS sales_by_brand
    FROM base
    GROUP BY consumer_id, brand
),
sales_product AS (
    SELECT consumer_id, product_id, SUM(total_amount) AS sales_by_product
    FROM base
    GROUP BY consumer_id, product_id
),
top_selling AS (
    SELECT
        consumer_id,
        product_id,
        SUM(quantity) AS total_qty,
        RANK() OVER (PARTITION BY consumer_id ORDER BY SUM(quantity) DESC) AS rnk
    FROM base
    GROUP BY consumer_id, product_id
),
top_selling_product AS (
    SELECT
        t.consumer_id,
        t.product_id AS top_product_id,
        p.product_name AS top_product_name
    FROM top_selling t
    JOIN {catalog}.{silver_schema}.cpg_product p ON t.product_id = p.product_id
    WHERE rnk = 1
),
returned_products AS (
    SELECT DISTINCT consumer_id, product_id
    FROM base
    WHERE c_oid IN (
        SELECT order_id
        FROM {catalog}.{silver_schema}.cpg_consumer_order
        WHERE order_status = 'Returned'
    )
),
low_stock AS (
    SELECT DISTINCT product_id
    FROM {catalog}.{silver_schema}.cpg_inventory
    WHERE quantity_on_hand < reorder_level
),
payment_method_usage AS (
    SELECT category, payment_method_usage
    FROM base
    GROUP BY category, payment_method_usage
),
monthly_sales AS (
    SELECT DATE_TRUNC('month', ordered_date) AS month, SUM(total_amount) AS monthly_sales
    FROM base
    GROUP BY DATE_TRUNC('month', ordered_date)
),
clv AS (
    SELECT consumer_id, SUM(total_amount) AS customer_lifetime_value
    FROM base
    GROUP BY consumer_id
),
repeat_purchase AS (
    SELECT 
        COUNT(DISTINCT consumer_id) FILTER (WHERE order_count > 1)::decimal / 
        NULLIF(COUNT(DISTINCT consumer_id), 0) AS repeat_purchase_rate
    FROM (
        SELECT consumer_id, COUNT(order_id) AS order_count
        FROM {catalog}.{silver_schema}.cpg_consumer_order
        GROUP BY consumer_id
    ) sub
)

-- New Customers Acquired (per month)
,new_customers AS (
    SELECT 
        DATE_TRUNC('month', registration_date) AS month,
        COUNT(DISTINCT consumer_id) AS new_customers
    FROM {catalog}.{silver_schema}.cpg_consumer
    GROUP BY DATE_TRUNC('month', registration_date)
)

-- Fulfillment Rate (delivered / total orders)
,fulfillment AS (
    SELECT 
        COUNT(*) FILTER (WHERE order_status = 'Delivered')::decimal / 
        NULLIF(COUNT(*),0) AS fulfillment_rate
    FROM {catalog}.{silver_schema}.cpg_consumer_order
)

-- Orders per Payment Method
,orders_per_payment AS (
    SELECT payment_method, COUNT(order_id) AS orders_per_payment
    FROM {catalog}.{silver_schema}.cpg_consumer_order
    GROUP BY payment_method
)

SELECT
    p.order_item_id,
    p.product_id,
    p.quantity,
    p.selling_price,
    p.margin,
    p.total_amount,
    p.profit_margin,
    p.s_oid,
    p.product_name,
    p.department,
    p.category,
    p.brand,
    p.retail_price,
    p.cost_price,
    p.release_date,
    p.c_oid,
    p.ordered_date,
    p.order_status,
    p.total_amount_order,
    p.payment_method_usage,
    p.consumer_id,
    p.consumer_name,
    p.gender,
    p.age,
    p.registration_date,
    p.is_active,
    p.city,
    p.state,
    p.country,
    p.location_name,
    p.inventory_id,
    p.quantity_on_hand,
    p.inventory_status,
    p.reorder_level,
    o.total_orders,
    o.total_sales_revenue,
    o.avg_order_value,
    sc.sales_by_category,
    sb.sales_by_brand,
    sp.sales_by_product,
    ts.top_product_id,
    ts.top_product_name,
    rp.repeat_purchase_rate,
    nc.new_customers,
    f.fulfillment_rate,
    op.orders_per_payment,
    CASE WHEN rp.product_id IS NOT NULL THEN 1 ELSE 0 END AS is_returned_product,
    CASE WHEN ls.product_id IS NOT NULL THEN 1 ELSE 0 END AS is_low_stock,
    pmu.payment_method_usage AS payment_method_category_usage,
    ms.month,
    ms.monthly_sales,
    c.customer_lifetime_value,
    tm.total_consumer_sales,
    ROUND(tm.average_order_value) AS global_average_order_value,
    tm.total_order_count,
    ROUND(of.order_frequency) AS avg_order_frequency_per_consumer,
    ac.active_consumer_count
FROM base p
LEFT JOIN repeat_purchase rp ON 1=1
LEFT JOIN new_customers nc ON DATE_TRUNC('month', p.ordered_date) = nc.month
LEFT JOIN fulfillment f ON 1=1
LEFT JOIN orders_per_payment op ON p.payment_method_usage = op.payment_method
LEFT JOIN orders_sales o ON p.consumer_id = o.consumer_id
LEFT JOIN sales_category sc ON p.consumer_id = sc.consumer_id AND p.category = sc.category
LEFT JOIN sales_brand sb ON p.consumer_id = sb.consumer_id AND p.brand = sb.brand
LEFT JOIN sales_product sp ON p.consumer_id = sp.consumer_id AND p.product_id = sp.product_id
LEFT JOIN top_selling_product ts ON p.consumer_id = ts.consumer_id
LEFT JOIN returned_products rp ON p.consumer_id = rp.consumer_id AND p.product_id = rp.product_id
LEFT JOIN low_stock ls ON p.product_id = ls.product_id
LEFT JOIN payment_method_usage pmu ON p.category = pmu.category AND p.payment_method_usage = pmu.payment_method_usage
LEFT JOIN monthly_sales ms ON DATE_TRUNC('month', p.ordered_date) = ms.month
LEFT JOIN clv c ON p.consumer_id = c.consumer_id
CROSS JOIN total_metrics tm
CROSS JOIN order_frequency of
CROSS JOIN active_consumers ac;
""")